In [17]:
from dotenv import load_dotenv

load_dotenv()

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
import random

import numpy as np


In [2]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [8]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    ques: str = Field(description="객관식으로 선택할 수 있는 문제")
    ans: str = Field(description="문제에 대한 정답")

In [10]:
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='C:/project4/chat/testDB')
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 선생님입니다. {context}를 기반으로 정확하게 {theme}에 대한 객관식으로 선택할 수 있는 문제와 문제에 대한 정답을 만들어주세요. 문제는 이의 제기가 일어나지 않게 '거리가 가까운 것은?', '거리가 먼 것은?'이라는 형태로 끝나야 합니다."),
        ("user", "#Format: {format_instructions}"),
    ]
)

In [15]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())
prompt

ChatPromptTemplate(input_variables=['context', 'theme'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"ques": {"description": "객관식으로 선택할 수 있는 문제", "title": "Ques", "type": "string"}, "ans": {"description": "문제에 대한 정답", "title": "Ans", "type": "string"}}, "required": ["ques", "ans"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'theme'], input_types={}, partial_variables={}, template="당신은 선생님입니다. {context}를 기반으로 정확하게 {theme}에 대한 객관식으로 선택할 수 있는 문제와 문제에 대한 정답을

In [ ]:
# page_content만 저장
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {'context': retriever|format_docs, 'theme': RunnablePassthrough()} # chat_prompt가 갖는 dict
    |prompt | model | parser)  # 체인을 구성합니다.

In [18]:
theme_list = ["해양쓰레기 종류", "해양쓰레기 발생원인", "해양쓰레기 현황", "해양쓰레기 피해 및 위험성"]
n = np.random.randint(0,len(theme_list))
ans = chain.invoke(theme_list[n])

In [19]:
ans

{'ques': '해양쓰레기로 인한 피해 중에서 거리가 가까운 것은?', 'ans': '선박사고의 10분의 1은 해양쓰레기 때문입니다.'}

In [38]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="'{ques}' 문제에 대한 오답을 3가지 만들어주세요. 오해의 소지가 있을 문구는 제외하여 오답을 만들주세요. 오답은 '{ans}'와 비슷한 형식으로 만들어주세요.\n{format_instructions}",
    input_variables=["ques", "ans"],
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

In [39]:
prompt.format(ques="해양쓰레기 종류에 대한 설명으로 알맞은 것은?", ans="해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.")

"'해양쓰레기 종류에 대한 설명으로 알맞은 것은?' 문제에 대한 오답을 3가지 만들어주세요. 오해의 소지가 있을 문구는 제외하여 오답을 만들주세요. 오답은 '해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.'와 비슷한 형식으로 만들어주세요.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"

In [40]:
chain = (
    RunnableMap({
        'ques': RunnablePassthrough(),
        'ans': RunnablePassthrough(),
    })
    |prompt | model | output_parser
)

In [47]:
ans = chain.invoke({'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?', 'ans': '어구와 낚시 용품'})

In [48]:
ans

["{'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?'",
 "'ans': '플라스틱 병'}",
 "{'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?'",
 "'ans': '종이컵'}",
 "{'ques': '해양쓰레기의 육상 기인 쓰레기가 아닌 것은?'",
 "'ans': '음료수 캔'}"]

In [30]:
ans1 = chain.invoke({"ques":"해양쓰레기 종류에 대한 설명으로 알맞은 것은?", "ans": "해양쓰레기는 육상과 해상 모두에서 발생할 수 있다.",})

In [31]:
ans1

["{'ques': '해양쓰레기 종류에 대한 설명으로 알맞은 것은?'",
 "'ans': '해양쓰레기는 오직 해상에서만 발생한다.'}",
 "{'ques': '해양쓰레기 종류에 대한 설명으로 알맞은 것은?'",
 "'ans': '해양쓰레기는 주로 플라스틱으로만 구성되어 있다.'}",
 "{'ques': '해양쓰레기 종류에 대한 설명으로 알맞은 것은?'",
 "'ans': '해양쓰레기는 자연적으로 분해되지 않는다.'}"]